## imports

In [14]:
import numpy as np
import pandas as pd
import copy

In [15]:
def write_as_log(title,data):
    '''
    Function can be used to write a log in the logs folder. 
    Data has to be an array 
    '''
    with open(f'results/Combining/{title}.txt','w') as temp:
        if type(data) == list:
            for row in data:
                temp.write(str(row))
                temp.write('\n')

## optimal greedy clustering

In [16]:
# laad omvormer
omvormer = np.load('omvormer.npy',allow_pickle='TRUE').item()

#load optimal greedy cautious threshold 0.8 --> greedy optimization 0.7
optimal = np.load("C://Users//lexfo//PycharmProjects//thesis_lex//results//Clustering//optimal_greedy.npy",allow_pickle='TRUE')
best_greedy_clustering = []
for i in range(len(optimal)):
    best_greedy_clustering.append(optimal[i])

In [17]:
clusters_omgevormd = [] 
for i in range(len(best_greedy_clustering)):
    clusters_omgevormd.append([omvormer.get(item,item) for item in best_greedy_clustering[i]])
# clusters_omgevormd

In [27]:
dataframe_best_greedy = pd.DataFrame.from_records(clusters_omgevormd)


<br>
<br>
<br>
<br>
<br><br>
<br>
<br>
<br>
<br><br>
<br>
<br>
<br>
<br>

## Open feature selection techniques

#### SVC-RFE

In [79]:
SVC_RFE = pd.read_csv('results/Feature_importance_ranking/SVC_RFE_Ranking.csv')
svc_rfe_Ranking = list(SVC_RFE['var'])

#### RF-RFE

In [67]:
RF_RFE = pd.read_csv('results/Feature_importance_ranking/RE_RFE_Ranking.csv')
rf_rfe_Ranking = list(RF_RFE['var'])

#### Boruta

In [127]:
Boruta = pd.read_csv('results/Feature_importance_ranking/Boruta_Ranking.csv')
Boruta.rename(columns={'Unnamed: 0':'var'}, inplace=True)
Boruta = Boruta.sort_values(by=['meanImp'], ascending = False)
Boruta = Boruta.reset_index(drop=True)
Boruta_Ranking = list(Boruta['var'])

#### XGBoost

In [68]:
XGBoost = pd.read_csv('results/Feature_importance_ranking/XGBoost_ranking_rerun.csv')
XGBoost_ranking = list(XGBoost['var'])

<br>
<br>
<br>
<br>
<br><br>
<br>
<br>
<br>
<br><br>
<br>
<br>
<br>
<br><br>
<br>
<br>
<br>
<br>

## combine clustering with feature ranking

In [7]:
def pick_1_per_cluster(dataframe_kluster,features):
    nieuwe = copy.deepcopy(dataframe_kluster)
    xgboost = features
    row_selected = []
    final_columns = []
    deleted_indexen = [] 
    counter = 0
    for j in xgboost:
        counter += 1 
        for i in range(len(nieuwe)):
            if i in row_selected: 
                pass
            elif j in nieuwe.loc[i].values:
                row_selected.append(i)
                print(i,j,counter)
                xgboost = [feature for feature in xgboost if feature not in nieuwe.loc[i].values]
                nieuwe.iloc[i].values[:] = 0
                final_columns.append(j)
            else: 
                pass
    return final_columns

In [31]:
def combine_ranking_with_clustering(best_clustering, name_of_fs_technique, feature_selection_ranking,top_x_of_ranking=0):
    if top_x_of_ranking > 1:
        ranking = feature_selection_ranking[:top_x_of_ranking]
        final_columns_after_combining = pick_1_per_cluster(best_clustering,ranking)
#         write_as_log(f'{name_of_fs_technique}_and_clustering_top_{top_x_of_ranking}', final_columns_after_combining)
        return final_columns_after_combining
    else:
        ranking = feature_selection_ranking
        final_columns_after_combining = pick_1_per_cluster(best_clustering,ranking)
#         write_as_log(f'{name_of_fs_technique}_and_clustering_total', final_columns_after_combining)
        return final_columns_after_combining

#### SVM-RFE

In [82]:
final_svc_rfe = combine_ranking_with_clustering(dataframe_best_greedy, 'svc_rfe', svc_rfe_Ranking)
print(len(final_rf_rfe))

1 ETHBTC__ticker_info__open 1
11 ETHBTC__technical_analysis_candles__macd 2
5 ETHBTC__ticker_info__quote_asset_volume 7
98 general_info__biggest_coin_streamer__BUSDUSDT 8
111 general_info__biggest_coin_streamer__ONTUSDT 14
93 general_info__exchange_info__Mexican_Peso 16
32 ETHBTC__ticker_info__number_trades 18
12 BTC__events_aggregator__NumSources_7_days 19
23 general_info__events_aggregator__NumSources_7_days 21
112 general_info__biggest_coin_streamer__XLMBTC 22
73 ETH__twitter_info_aggregated__total_followers 23
109 general_info__stock_exchange_index__KOSPI_Composite_Index 24
88 ETH__events_aggregator__GoldsteinScale_7_days 26
75 BTC__events_aggregator__NumMentions_1_hour 30
114 general_info__biggest_coin_streamer__ATOMUSDT 32
26 general_info__hash_rate_streamer_bitcoin__btc_sent_24 38
78 general_info__twitter_info_aggregated__polarity_uncleaned 39
97 general_info__events_aggregator__count_7_days 40
34 general_info__biggest_coin_streamer__WAVESBTC 43
103 general_info__biggest_coin_st

In [83]:
final_columns_svc_rfe = SVC_RFE[~SVC_RFE['var'].isin(final_svc_rfe) == False]

In [84]:
final_columns_svc_rfe = final_columns_svc_rfe.reset_index(drop=True)

In [86]:
# final_columns_svc_rfe.to_csv(r'results/Combining/final_columns_svc_rfe.csv', index = False)

#### RF-RFE

In [55]:
final_rf_rfe = combine_ranking_with_clustering(dataframe_best_greedy, 'rf_rfe', rf_rfe_Ranking)
print(len(final_rf_rfe))

89 ETHBTC__technical_analysis_candles__rsi 1
19 ETHBTC__technical_analysis_candles__macdhist 2
11 ETHBTC__technical_analysis_candles__macd 3
5 ETHBTC__ticker_info__taker_buy_asset_volume 4
32 ETHBTC__ticker_info__number_trades 9
79 ETHBTC__ticker_info__askQty 10
103 ETHBTC__ticker_info__bidQty 11
10 ETH__twitter_info_aggregated__relative_subjectivity_followers 12
86 general_info__twitter_info_aggregated__relative_subjectivity_friends 13
48 BTC__twitter_info_aggregated__relative_polarity_followers 14
85 ETH__twitter_info_aggregated__relative_polarity_followers 15
73 ETH__twitter_info_aggregated__total_followers 16
57 BTC__twitter_info_aggregated__relative_subjectivity_followers 17
52 general_info__twitter_info_aggregated__relative_subjectivity_followers 18
51 ETH__twitter_info_aggregated__polarity_self_cleaned 19
76 BTC__twitter_info_aggregated__relative_polarity_friends 20
59 general_info__twitter_info_aggregated__relative_polarity_followers 21
31 general_info__hash_rate_ethereum__gasU

In [60]:
final_columns_rf_rfe = RF_RFE[~RF_RFE['var'].isin(final_rf_rfe) == False]
    

In [61]:
final_columns_rf_rfe = final_columns_rf_rfe.reset_index(drop=True)

In [63]:
# final_columns_rf_rfe.to_csv(r'results/Combining/final_columns_rf_rfe.csv', index = False)

#### Boruta

In [129]:
final_boruta = combine_ranking_with_clustering(dataframe_best_greedy, 'Boruta', Boruta_Ranking)
print(len(final_boruta))

89 ETHBTC__technical_analysis_candles__rsi 1
5 ETHBTC__ticker_info__taker_buy_asset_volume 2
19 ETHBTC__technical_analysis_candles__macdhist 4
32 ETHBTC__ticker_info__number_trades 7
11 ETHBTC__technical_analysis_candles__macd 8
79 ETHBTC__ticker_info__askQty 10
1 ETHBTC__ticker_info__open 11
62 general_info__biggest_coin_streamer__ENJBTC 13
74 general_info__biggest_coin_streamer__ETHBTC 14
101 general_info__biggest_coin_streamer__XTZUSDT 19
93 general_info__biggest_coin_streamer__LINKUSDT 20
114 general_info__biggest_coin_streamer__ETCBTC 22
97 general_info__biggest_coin_streamer__AIONBTC 25
105 general_info__exchange_info__Lumens 29
109 general_info__biggest_coin_streamer__ETCUSDT 34
104 general_info__exchange_info__XRP 36
77 general_info__biggest_coin_streamer__XMRBTC 38
111 general_info__biggest_coin_streamer__NEBLBTC 43
99 general_info__biggest_coin_streamer__XRPBTC 48
4 general_info__biggest_coin_streamer__EOSBTC 51
92 general_info__exchange_info__Bitcoin_Cash 55
86 general_info_

In [130]:
final_columns_boruta = Boruta[~Boruta['var'].isin(final_boruta) == False]

In [132]:
final_columns_boruta = final_columns_boruta.reset_index(drop=True)

In [134]:
# final_columns_boruta.to_csv(r'results/Combining/final_columns_boruta.csv', index = False)

#### XGBoost

In [70]:
final_XGBoost = combine_ranking_with_clustering(dataframe_best_greedy, 'XGBoost2', XGBoost_ranking)

89 ETHBTC__technical_analysis_candles__rsi 1
19 ETHBTC__technical_analysis_candles__macdhist 2
11 ETHBTC__technical_analysis_candles__macd 3
1 ETHBTC__ticker_info__open 4
103 ETHBTC__ticker_info__bidQty 6
79 ETHBTC__ticker_info__askQty 7
10 ETH__twitter_info_aggregated__relative_subjectivity_followers 8
48 BTC__twitter_info_aggregated__relative_polarity_followers 9
67 ETH__twitter_info_aggregated__relative_subjectivity_friends 10
57 BTC__twitter_info_aggregated__relative_subjectivity_followers 11
73 ETH__twitter_info_aggregated__total_followers 12
85 ETH__twitter_info_aggregated__relative_polarity_followers 13
52 general_info__twitter_info_aggregated__relative_subjectivity_followers 14
59 general_info__twitter_info_aggregated__relative_polarity_followers 15
31 general_info__hash_rate_ethereum__gasUsed 16
100 BTC__twitter_info_aggregated__relative_subjectivity_friends 17
56 general_info__hash_rate_ethereum__gasLimit 18
51 ETH__twitter_info_aggregated__polarity_self_cleaned 19
8 ETH__twi

In [74]:
final_columns_XGBoost = XGBoost[~XGBoost['var'].isin(final_XGBoost) == False]

In [76]:
final_columns_XGBoost = final_columns_XGBoost.reset_index(drop=True)

In [77]:
# final_columns_XGBoost.to_csv(r'results/Combining/final_columns_XGBoost.csv', index = False)